In [5]:
import psycopg2

try:
    connection = psycopg2.connect(
        user="user",
        port="port",
        host="localhost",
        database="name"
    )
    
    cursor = connection.cursor()
    
    # The table sentiment_analysis_data_amzn contains tens of thousands of news articles about AMZN
    select_query1 = "SELECT * FROM sentiment_analysis_data_amzn;"

    cursor.execute(select_query1)
    
    # Fetch all records
    records_amzn = cursor.fetchall()
    print(records_amzn[700][3])
    
except Exception as error:
    print(f"Failed to fetch records: {error}")

finally:
    # Close the connection
    if connection:
        cursor.close()
        connection.close()
        print("Connection closed.")

Jim Cramer, in a special edition of TheStreet.com TV's Wall Street Confidential Web video, explained what not to do when shorting a stock. Cramer said that whether you are a professional or amateur, do not use only common stock to short. "It doesn't work," he said. For people who are addicted to shorting common stock, Cramer suggested buying the Whirlpool(WHR:NYSE) 85 calls as insurance and shorting common against it. Do not use regular common stock to short "in this kind of blowoff to the upside," Cramer stressed. Or, you can just buy puts instead. "If you're going to short, you can only do it with puts, you cannot do it with common," Cramer said. (Shorting is borrowing stock and selling it for the proceeds, with the expectation it will go down and eventually can be bought back at a lower price. The repurchased shares then must be returned to the lender. A call is an option to buy a stock at a predetermined price by a certain date; a put is the opposite, to sell at a specific price.) 

In [4]:
connection = psycopg2.connect(
    user="user",
    port="port",
    host="localhost",
    database="name"
)

cursor = connection.cursor()

drop_table_query = "DROP TABLE IF EXISTS sentiment_data_amzn;"
cursor.execute(drop_table_query)
connection.commit()
print("Table dropped successfully.")

create_table_query = '''CREATE TABLE sentiment_data_amzn (
    ID SERIAL PRIMARY KEY,
    Title TEXT,
    Time_Published TEXT,
    Sentiment_Score REAL
); '''

cursor.execute(create_table_query)
connection.commit()
print("Table Created Successfully")

Table dropped successfully.
Table Created Successfully


In [7]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from nltk.tokenize import sent_tokenize
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

connection = psycopg2.connect(
    user="user",
    port="port",
    host="localhost",
    database="name"
)

cursor = connection.cursor()

# Load the model and tokenizer
model_name = "nickmuchi/deberta-v3-base-finetuned-finance-text-classification"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)  # Moved to device
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Preprocessing function
def preprocess(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    return inputs

# Inference function for a single sentence
def get_sentiment(text):
    inputs = preprocess(text).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    label_id = torch.argmax(probabilities).item()
    return label_id

# Your articles
article_text = [record[3] for record in records_amzn]
article_date = [record[2] for record in records_amzn]
article_title = [record[1] for record in records_amzn]

# Iterate over articles
for i, (article, date, title) in enumerate(zip(article_text, article_date, article_title)):
    sentences = sent_tokenize(article)  # Split the article into sentences
    sentiment_scores = [get_sentiment(sentence) for sentence in sentences]  # Get sentiment for each sentence
    avg_sentiment = sum(sentiment_scores) / len(sentiment_scores)  # Average the sentiment scores
    cursor.execute("INSERT INTO sentiment_data_amzn (ID, Title, Sentiment_Score, Time_Published) VALUES (%s, %s, %s, %s)",
                   (i, title, avg_sentiment, date))
    connection.commit()
    print(f"Average Sentiment of Article {i+1} is {avg_sentiment} at {date} for {title}")

Average Sentiment of Article 1 is 0.9166666666666666 at January 31, 2007 Wednesday 11:01 PM EST
Average Sentiment of Article 2 is 1.3636363636363635 at January 22, 2007 Monday 10:39 AM EST
Average Sentiment of Article 3 is 0.16666666666666666 at January 3, 2007 Wednesday 10:23 AM EST
Average Sentiment of Article 4 is 1.5454545454545454 at January 3, 2007 Wednesday 12:24 PM EST
Average Sentiment of Article 5 is 1.1 at January 29, 2007 Monday 1:30 PM GMT
Average Sentiment of Article 6 is 1.0444444444444445 at January 18, 2007 Thursday 14:46 PM EST
Average Sentiment of Article 7 is 1.0263157894736843 at January 16, 2007 Tuesday 15:55 PM EST
Average Sentiment of Article 8 is 1.1578947368421053 at January 24, 2007 Wednesday 4:46 PM EST
Average Sentiment of Article 9 is 1.0416666666666667 at January 16, 2007 Tuesday
Average Sentiment of Article 10 is 1.14 at January 29, 2007 Monday 07:45 AM EST
Average Sentiment of Article 11 is 0.9 at January 27, 2007 Saturday 9:55 AM EST
Average Sentiment 

KeyboardInterrupt: 